# Biodiversity in National Parks
This is a project from CodeCademy's 'Data Scientist' Pathway, and is the 'Data Analysis Porfolio Project' capstone. Please see below for a brief, project description:
> For this project, you will interpret data from the National Parks Service about endangered species in different parks.
>
> You will perform some data analysis on the conservation statuses of these species and investigate if there are any patterns or themes to the types of species that become endangered. During this project, you will analyze, clean up, and plot data as well as pose questions and seek to answer them in a meaningful way.
>
> After you perform your analysis, you will share your findings about the National Park Service.

## Overview
In this project, we will interpret data about endangered species in different parks, inspect and clean the data, and investigate any patterns or themes to the types of species that become endangered.

## Setup & EDA

In [1]:
# importing libraries
import pandas as pd

In [2]:
# loading data
species_info = pd.read_csv('biodiversity_starter/species_info.csv')
observations = pd.read_csv('biodiversity_starter/observations.csv')

### Inspecting & cleaning

#### `species_info`

In [3]:
species_info.head()

,category,scientific_name,common_names,conservation_status
0,Mammal,Clethrionomys gapperi gapperi,Gapper's Red-Backed Vole,NaN
1,Mammal,Bos bison,"American Bison, Bison",NaN
2,Mammal,Bos taurus,"Aurochs, Aurochs, Domestic Cattle (Feral), Dom...",NaN
3,Mammal,Ovis aries,"Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)",NaN
4,Mammal,Cervus elaphus,Wapiti Or Elk,NaN


In [4]:
species_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  191 non-null    object
dtypes: object(4)
memory usage: 182.1+ KB


In [5]:
for col in species_info.columns:
    display(col, species_info[col].value_counts())

'category'

Vascular Plant       4470
Bird                  521
Nonvascular Plant     333
Mammal                214
Fish                  127
Amphibian              80
Reptile                79
Name: category, dtype: int64

'scientific_name'

Puma concolor                 3
Columba livia                 3
Procyon lotor                 3
Myotis lucifugus              3
Streptopelia decaocto         3
                             ..
Carex luzulina var. ablata    1
Castilleja foliolosa          1
Piranga ludoviciana           1
Tringa melanoleuca            1
Blindia acuta                 1
Name: scientific_name, Length: 5541, dtype: int64

'common_names'

Dicranum Moss                                                                                                                7
Brachythecium Moss                                                                                                           7
Bryum Moss                                                                                                                   6
Sphagnum                                                                                                                     6
Panic Grass                                                                                                                  6
                                                                                                                            ..
Jim Hill Mustard, Tall Hedge-Mustard, Tall Mustard, Tall Tumblemustard, Tumble Mustard, Tumblemustard, Tumbleweed Mustard    1
Hairy Indian Paintbrush, Slender Paintbrush                                                                    

'conservation_status'

Species of Concern    161
Endangered             16
Threatened             10
In Recovery             4
Name: conservation_status, dtype: int64

##### Missing
`conversation_status` is the only column with missing values but are difficult to impute as categories do not reflect those of the [IUCN Red List](https://www.iucnredlist.org/#:~:text=It%20divides%20species%20into%20nine,in%20the%20Wild%20and%20Extinct.). Also, would not be dropping those species, as their presence would likely be important in later analysis to indicate their existence in the habitat scope of the data. 

No further actions.

In [6]:
species_info.isna().sum()

category                  0
scientific_name           0
common_names              0
conservation_status    5633
dtype: int64

In [7]:
species_info[species_info.isnull().any(axis=1)]

,category,scientific_name,common_names,conservation_status
0,Mammal,Clethrionomys gapperi gapperi,Gapper's Red-Backed Vole,NaN
1,Mammal,Bos bison,"American Bison, Bison",NaN
2,Mammal,Bos taurus,"Aurochs, Aurochs, Domestic Cattle (Feral), Dom...",NaN
3,Mammal,Ovis aries,"Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)",NaN
4,Mammal,Cervus elaphus,Wapiti Or Elk,NaN
...,...,...,...,...
5819,Vascular Plant,Solanum parishii,Parish's Nightshade,NaN
5820,Vascular Plant,Solanum xanti,"Chaparral Nightshade, Purple Nightshade",NaN
5821,Vascular Plant,Parthenocissus vitacea,"Thicket Creeper, Virginia Creeper, Woodbine",NaN
5822,Vascular Plant,Vitis californica,"California Grape, California Wild Grape",NaN


##### Duplication

Initially, no duplication across all fields:

In [8]:
species_info[species_info.duplicated()]

,category,scientific_name,common_names,conservation_status


However, from above, there were multiple `scientific_name`s in the `.value_count()`, indicating potential duplication. Each row should be a unique species.

How to handle duplicates will depend on the nature of the duplicates.

**Category 1: duplicated names of species**

Where both the scientific and common names of a species are duplicated:

In [9]:
species_names_dup = species_info[species_info.duplicated(subset=['scientific_name', 'common_names'], keep=False)]\
                        .sort_values(['scientific_name', 'category'])

species_names_dup.scientific_name.value_counts()

Oncorhynchus mykiss    2
Canis lupus            2
Name: scientific_name, dtype: int64

In [10]:
species_names_dup

,category,scientific_name,common_names,conservation_status
3020,Mammal,Canis lupus,"Gray Wolf, Wolf",In Recovery
4448,Mammal,Canis lupus,"Gray Wolf, Wolf",Endangered
560,Fish,Oncorhynchus mykiss,Rainbow Trout,NaN
3283,Fish,Oncorhynchus mykiss,Rainbow Trout,Threatened


***Canis lupus***

As of 4 Jan 2021, the Fish and Wildlife Service ruled that *Canis lupus* is not classified as threatened or endangered, but the Mexican wolf subspecies (*Canis lupus baileyi*) is endangered ([Source](https://www.federalregister.gov/documents/2020/11/03/2020-24171/endangered-and-threatened-wildlife-and-plants-removing-the-gray-wolf-canis-lupus-from-the-list-of)). This likely explains the two separate entries for *Canis lupus*, with difference `conservation_status` values.

Because each row is a species, and [a subspecies cannot be recognised independently](https://en.wikipedia.org/wiki/Subspecies#Nominotypical_subspecies_and_subspecies_autonyms), we should not add subspecies suffixes to `scientific_name` to distinguish. Instead, we will add further `common_names`:

In [11]:
# creating cleaned `species_info` df
species_info_cleaned = species_info

species_info_cleaned.at[4448,'common_names'] = 'Mexican Wolf, Lobo'

species_info_cleaned[species_info_cleaned.scientific_name=='Canis lupus']

,category,scientific_name,common_names,conservation_status
8,Mammal,Canis lupus,Gray Wolf,Endangered
3020,Mammal,Canis lupus,"Gray Wolf, Wolf",In Recovery
4448,Mammal,Canis lupus,"Mexican Wolf, Lobo",Endangered


From above, we will also drop the *Canis lupus* entry at index 8:

In [12]:
species_info_cleaned = species_info_cleaned.drop(8)

species_info_cleaned[species_info_cleaned.scientific_name=='Canis lupus']

,category,scientific_name,common_names,conservation_status
3020,Mammal,Canis lupus,"Gray Wolf, Wolf",In Recovery
4448,Mammal,Canis lupus,"Mexican Wolf, Lobo",Endangered


***Oncorhynchus mykiss***

As of 17 Feb 2022, the [NOAA Fisheries](https://www.fisheries.noaa.gov//species/steelhead-trout#overview) (US Dept of Commerce) has listed one distinct population segment ('DPS') (the 'Southern California DPS') as endangered, and 10 DPS and 1 experimental, non-essential population as threatened. This likely explains the duplication.

Similarly to above, DPS suffixes should not be added to `scientific_name`. To distinguish between the two rows, we will:
1. Update the `common_names`, and
2. Update the `conservation_status`

In [13]:
species_info_cleaned[species_info_cleaned.scientific_name=='Oncorhynchus mykiss']

,category,scientific_name,common_names,conservation_status
560,Fish,Oncorhynchus mykiss,Rainbow Trout,NaN
3283,Fish,Oncorhynchus mykiss,Rainbow Trout,Threatened


In [14]:
species_info_cleaned.at[560,'common_names'] = 'Southern California Steelhead'
species_info_cleaned.at[560,'conservation_status'] = 'Endangered'
species_info_cleaned.at[3283, 'common_names'] = 'Rainbow Trout, Steelhead Trout, Redband Trout'

species_info_cleaned[species_info_cleaned.scientific_name=='Oncorhynchus mykiss']

,category,scientific_name,common_names,conservation_status
560,Fish,Oncorhynchus mykiss,Southern California Steelhead,Endangered
3283,Fish,Oncorhynchus mykiss,"Rainbow Trout, Steelhead Trout, Redband Trout",Threatened


**Category 2: same scientific name and conservation status**
If the species has the same scientific name and conservation status, it likely means there are subspecies of other subdivisions of the species. However, because the scope of the analysis will be on the `conservation_status`, we can drop these duplicates as those subdivisions are unlikely material.

In [15]:
species_scinames_dup = species_info_cleaned[species_info_cleaned.duplicated(subset=['scientific_name', 'conservation_status'],
                                                                            keep=False)]\
                        .sort_values('scientific_name')


# species_scinames_dup.scientific_name.unique()
species_scinames_dup

,category,scientific_name,common_names,conservation_status
5553,Vascular Plant,Agrostis capillaris,"Colonial Bent, Colonial Bentgrass",NaN
2132,Vascular Plant,Agrostis capillaris,Rhode Island Bent,NaN
5554,Vascular Plant,Agrostis gigantea,"Black Bent, Redtop, Water Bentgrass",NaN
2134,Vascular Plant,Agrostis gigantea,Redtop,NaN
4178,Vascular Plant,Agrostis mertensii,"Arctic Bentgrass, Northern Bentgrass",NaN
...,...,...,...,...
5643,Vascular Plant,Vulpia myuros,"Foxtail Fescue, Rattail Fescue, Rat-Tail Fescu...",NaN
4290,Vascular Plant,Vulpia octoflora,"Eight-Flower Six-Weeks Grass, Pullout Grass, S...",NaN
2331,Vascular Plant,Vulpia octoflora,Annual Fescue,NaN
632,Vascular Plant,Zizia aptera,Golden Alexanders,NaN


In [16]:
species_info_cleaned = species_info_cleaned\
                        .drop_duplicates(subset=['scientific_name', 'conservation_status'])

# species_info_cleaned.shape
species_info_cleaned[species_info_cleaned\
                     .duplicated(subset=['scientific_name', 'conservation_status'],
                                 keep=False)]

,category,scientific_name,common_names,conservation_status


#### `observations` DataFrame

In [17]:
observations.head()

,scientific_name,park_name,observations
0,Vicia benghalensis,Great Smoky Mountains National Park,68
1,Neovison vison,Great Smoky Mountains National Park,77
2,Prunus subcordata,Yosemite National Park,138
3,Abutilon theophrasti,Bryce National Park,84
4,Githopsis specularioides,Great Smoky Mountains National Park,85


In [18]:
observations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB


In [19]:
observations.describe()

,observations
count,23296.000000
mean,142.287904
std,69.890532
min,9.000000
25%,86.000000
50%,124.000000
75%,195.000000
max,321.000000


In [20]:
observations.scientific_name.value_counts()

Castor canadensis              12
Canis lupus                    12
Holcus lanatus                 12
Columba livia                  12
Myotis lucifugus               12
                               ..
Viola sheltonii                 4
Phacelia quickii                4
Elymus hystrix var. hystrix     4
Cardamine diphylla              4
Hemidactylium scutatum          4
Name: scientific_name, Length: 5541, dtype: int64

In [21]:
observations.park_name.value_counts()

Yosemite National Park                 5824
Bryce National Park                    5824
Yellowstone National Park              5824
Great Smoky Mountains National Park    5824
Name: park_name, dtype: int64

##### Missing
There are no missing values.

In [22]:
observations.isna().sum()

scientific_name    0
park_name          0
observations       0
dtype: int64

##### Duplicates
For a given park, some species had multiple observations. 1060 species were observed twice at the same park, and 36 were observed thrice at the same park (21,068 species were only observed once at the same park). 

It is possible that these observations were taken at different times, under different conditions. From the `species_info` dataset, it is also possible that the same scientific name refers to different subspecies. However, without these attributes in the dataset, they are difficult to impute. 

Instead, we can consider all the duplicates by taking the mean of each species, at each park.

In [23]:
# showing all duplicated rows where the same species, at the same park, was observed on multiple occasions
observations_dup = observations[observations.duplicated(subset=['scientific_name', 'park_name'], keep=False)]\
                        .sort_values(['scientific_name', 'park_name'])

observations_dup

,scientific_name,park_name,observations
3993,Agrostis capillaris,Bryce National Park,103
4864,Agrostis capillaris,Bryce National Park,105
792,Agrostis capillaris,Great Smoky Mountains National Park,84
17428,Agrostis capillaris,Great Smoky Mountains National Park,97
6166,Agrostis capillaris,Yellowstone National Park,241
...,...,...,...
11138,Zizia aptera,Great Smoky Mountains National Park,41
6963,Zizia aptera,Yellowstone National Park,257
17162,Zizia aptera,Yellowstone National Park,269
8445,Zizia aptera,Yosemite National Park,123


In [24]:
# for the species observed at the same park, multiple times, finding out the number of times they were observed:
observations_dup_count = observations.groupby(['scientific_name', 'park_name']).count()

# observations_dup_count[observations_dup_count.observations!=2]
observations_dup_count.observations.value_counts()

1    21068
2     1060
3       36
Name: observations, dtype: int64

In [25]:
# taking the mean observations for each species, at each park
observations_summarised = observations\
                            .groupby(['scientific_name', 'park_name']).observations.mean()\
                            .reset_index()\
                            .rename({'observations': 'mean_observations'}, axis='columns') # renaming column

observations_summarised

,scientific_name,park_name,mean_observations
0,Abies bifolia,Bryce National Park,109.0
1,Abies bifolia,Great Smoky Mountains National Park,72.0
2,Abies bifolia,Yellowstone National Park,215.0
3,Abies bifolia,Yosemite National Park,136.0
4,Abies concolor,Bryce National Park,83.0
...,...,...,...
22159,Zygodon viridissimus,Yosemite National Park,159.0
22160,Zygodon viridissimus var. rupestris,Bryce National Park,102.0
22161,Zygodon viridissimus var. rupestris,Great Smoky Mountains National Park,102.0
22162,Zygodon viridissimus var. rupestris,Yellowstone National Park,237.0


## Exploring Data

### Category most prevalent of 'endangered' status
(May want to also look at the other status too)

### Relationship between 'endangered' status and observations?

### Within each category (mammals, fish etc.), which has the highest % of endangered, threatened, etc.?

### Using observations (and definitions of 'endangered'), predicting the next endangered species